In [21]:
import os

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
BASE_PATH = "/content/drive/MyDrive/makemore_course/"

In [24]:
NAMES = os.path.join(BASE_PATH, 'names.txt')

In [25]:
names = []
for f in open(NAMES, "r"):
  names.append(f[:-1])

In [26]:
names[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']



**E01**: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

In [27]:
import torch
import torch.nn.functional as F
import itertools
import random

In [28]:
alphabet = list(map(chr, range(97, 123)))
alphabet[:5]

['a', 'b', 'c', 'd', 'e']

In [29]:
list(itertools.product(alphabet[:5],repeat=2))

[('a', 'a'),
 ('a', 'b'),
 ('a', 'c'),
 ('a', 'd'),
 ('a', 'e'),
 ('b', 'a'),
 ('b', 'b'),
 ('b', 'c'),
 ('b', 'd'),
 ('b', 'e'),
 ('c', 'a'),
 ('c', 'b'),
 ('c', 'c'),
 ('c', 'd'),
 ('c', 'e'),
 ('d', 'a'),
 ('d', 'b'),
 ('d', 'c'),
 ('d', 'd'),
 ('d', 'e'),
 ('e', 'a'),
 ('e', 'b'),
 ('e', 'c'),
 ('e', 'd'),
 ('e', 'e')]

In [30]:
# Possible combinations of two chars (underscore)
list(map(lambda x : x[0]+x[1], itertools.product(alphabet[:3],repeat = 2))) 

['aa', 'ab', 'ac', 'ba', 'bb', 'bc', 'ca', 'cb', 'cc']

In [31]:
comb2 = list(map(lambda x : x[0]+x[1], itertools.product(alphabet,repeat = 2))) 
random.sample(comb2,10)

['vl', 'cp', 'hi', 'ip', 'ut', 'zi', 'lk', 'ez', 'lx', 'xs']

In [32]:
# we are missing the start end token ".", let's add it.
# we can have it either in front or in the back of any letter.
s = list(map(lambda x: x[0]+x[1], zip(["." for _ in range(26)],alphabet)))
e = list(map(lambda x: x[0]+x[1], zip(alphabet, ["." for _ in range(26)])))

In [33]:
random.sample(s,3)

['.r', '.p', '.z']

In [34]:
random.sample(e,3)

['k.', 'y.', 'm.']

In [35]:
# add to our list of possible combinations:
comb2 += s
comb2 += e

In [36]:
print(f"{len(comb2)} combiantions!")
print(f"Sanity check: combs of letters + start + end = 26 x 26  + 26 + 26 = {26*26 + 26 + 26}")

728 combiantions!
Sanity check: combs of letters + start + end = 26 x 26  + 26 + 26 = 728


In [37]:
# So what we have now is the possible first part of our trigram,
# which consist of 2 chars
# We need a way of encoding it.
chars_to_idx = {chars:idx for idx, chars in enumerate(comb2)}
chars_to_idx["ax"]

23

In [38]:
# Also we want a mapping back:
idx_to_chars = {idx:chars for chars, idx in chars_to_idx.items()}
idx_to_chars[23]

'ax'

In [39]:
# analog for the second part of the trigram.
char_to_idx = {char:idx for idx, char in enumerate(['.'] + alphabet)}
char_to_idx["x"]

24

In [40]:
idx_to_char = {idx:char for char, idx in char_to_idx.items()}
idx_to_char[23]

'w'

In [41]:
# Consider following list of words
words = ["dagobert", "donald", "gustav"]
words

['dagobert', 'donald', 'gustav']

In [42]:
# possible first part of trigram
list(map(lambda x: x[0]+x[1], (zip("." + words[0], words[0][:]))))

['.d', 'da', 'ag', 'go', 'ob', 'be', 'er', 'rt']

In [43]:
# possible last part of trigram
[x for x in words[0][2:] + "."]

['g', 'o', 'b', 'e', 'r', 't', '.']

In [44]:
# lets print out all trigrams in the above words!
for word in words:
  for x,y in zip(map(lambda x: x[0]+x[1], 
                     (zip("." + word, word[:]))),[x for x in word[1:] + "."]):
    print(x,y)

.d a
da g
ag o
go b
ob e
be r
er t
rt .
.d o
do n
on a
na l
al d
ld .
.g u
gu s
us t
st a
ta v
av .


In [45]:
# we can convert to int with the dicts from above:
# lets print out all trigrams in the above words!
for word in words:
  for x,y in zip(map(lambda x: x[0]+x[1], 
                     (zip("." + word, word[:]))),[x for x in word[1:] + "."]):
    x_i = chars_to_idx[x]
    y_i = char_to_idx[y]
    print(x_i, y_i)

679 1
78 7
6 15
170 2
365 5
30 18
121 20
461 0
679 15
92 14
377 1
338 12
11 4
289 0
682 21
176 19
538 20
487 1
494 22
21 0


In [46]:
def create_dataset(words):
  x_s = []
  y_s = []
  for word in words:
    for x,y in zip(map(lambda x: x[0]+x[1], 
                      (zip("." + word, word[:]))),[x for x in word[1:] + "."]):
      x_i = chars_to_idx[x]
      y_i = char_to_idx[y]
      x_s.append(x_i)
      y_s.append(y_i)
  return torch.tensor(x_s), torch.tensor(y_s)

In [47]:
x_s, y_s = create_dataset(words)

In [48]:
x_enc = F.one_hot(x_s, num_classes = 728).float()
x_enc[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

In [82]:
def init_weight():
  return torch.randn((728,27), requires_grad=True)

In [50]:
W = init_weight()

In [51]:
W.shape

torch.Size([728, 26])

In [52]:
logit = x_enc@W 
counts = logit.exp()
probs = counts/counts.sum(1,keepdims=True)

In [53]:
for i, y in enumerate(y_s):
  print(f"prob for real label predicted by model = {probs[i,y].item():.4f}")

prob for real label predicted by model = 0.0055
prob for real label predicted by model = 0.1177
prob for real label predicted by model = 0.0085
prob for real label predicted by model = 0.0964
prob for real label predicted by model = 0.0165
prob for real label predicted by model = 0.0925
prob for real label predicted by model = 0.0178
prob for real label predicted by model = 0.0262
prob for real label predicted by model = 0.0419
prob for real label predicted by model = 0.0596
prob for real label predicted by model = 0.0688
prob for real label predicted by model = 0.0128
prob for real label predicted by model = 0.0583
prob for real label predicted by model = 0.0071
prob for real label predicted by model = 0.0377
prob for real label predicted by model = 0.0190
prob for real label predicted by model = 0.0211
prob for real label predicted by model = 0.0203
prob for real label predicted by model = 0.0053
prob for real label predicted by model = 0.0173


In [54]:
# vectorized version:
probs[torch.arange(20),y_s]

tensor([0.0055, 0.1177, 0.0085, 0.0964, 0.0165, 0.0925, 0.0178, 0.0262, 0.0419,
        0.0596, 0.0688, 0.0128, 0.0583, 0.0071, 0.0377, 0.0190, 0.0211, 0.0203,
        0.0053, 0.0173], grad_fn=<IndexBackward0>)

In [55]:
# Lets encode the names
x_s, y_s = create_dataset(names)

In [157]:
x_enc = F.one_hot(x_s, num_classes = 728).float()
x_enc[0]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 

Steps for training sumarized:

In [115]:
# init Weights:
W = init_weight()

In [59]:
# calc log counts:
def calc_log_counts(W):
  return x_enc@W

In [62]:
# calc counts:
def calc_counts(logits):
  return logits.exp()

In [70]:
# calc probs:
def calc_probs(counts):
  return counts/counts.sum(1, keepdims=True)

In [125]:
# calculate the loss:
# In we are interested that the joined probs are close to 1
# take the log of joints probs as log(a*b) = log(a)+log(b)
# in each step we are interested in the log(prob) given the labels.
# As log(1) = 0 our goal and bad loss is high positive number multiply by -1

def calc_log_prob(probs, y):
  n = len(y)
  log_prob = -probs[torch.arange(n),y].log().mean()
  return log_prob

In [126]:
# A step towards the right solution ;-):
def step(W, lr):
  # zero out the gradient:
  W.grad = None
  logits = calc_log_counts(W)
  counts = calc_counts(logits)
  probs = calc_probs(counts)
  loss = calc_log_prob(probs, y_s)
  # accumalate gradients
  loss.backward()
  # update!
  with torch.no_grad():
    W -= lr * W.grad
  return loss, W

In [127]:
step(W,0.1)

(tensor(2.4479, grad_fn=<NegBackward0>),
 tensor([[ 0.9899, -1.0495, -0.5697,  ..., -1.7213, -0.2088, -0.2223],
         [ 1.1059,  1.0014,  0.6420,  ..., -0.8194, -0.3561, -1.4741],
         [-0.5498,  1.2269, -0.7097,  ..., -0.4661,  0.7803, -0.6377],
         ...,
         [ 0.8047, -0.7910, -1.5465,  ...,  1.6799, -1.2296, -0.4085],
         [ 0.5384, -1.1200,  0.4215,  ..., -0.8040,  2.6702,  0.1057],
         [ 0.0733,  0.2941, -1.0290,  ..., -1.5909,  1.5253, -0.3907]],
        requires_grad=True))

In [184]:
# run a whole training loop:
W = init_weight()
x_enc = F.one_hot(x_s, num_classes = 728).float()
for k in range(300):
  loss, W = step(W, 350)
  if k%10 == 0:
    print(f"epoch = {k}")
    print(f"current loss = {loss.item()}")

epoch = 0
current loss = 3.807774782180786
epoch = 10
current loss = 2.4857935905456543
epoch = 20
current loss = 2.326247453689575
epoch = 30
current loss = 2.2575438022613525
epoch = 40
current loss = 2.2184062004089355
epoch = 50
current loss = 2.192843437194824
epoch = 60
current loss = 2.1748046875
epoch = 70
current loss = 2.1614112854003906
epoch = 80
current loss = 2.1510822772979736
epoch = 90
current loss = 2.1428706645965576
epoch = 100
current loss = 2.136178493499756
epoch = 110
current loss = 2.1306140422821045
epoch = 120
current loss = 2.125908374786377
epoch = 130
current loss = 2.1218738555908203
epoch = 140
current loss = 2.118373394012451
epoch = 150
current loss = 2.1153056621551514
epoch = 160
current loss = 2.1125943660736084
epoch = 170
current loss = 2.1101794242858887
epoch = 180
current loss = 2.108013868331909
epoch = 190
current loss = 2.10606050491333
epoch = 200
current loss = 2.1042888164520264
epoch = 210
current loss = 2.1026740074157715
epoch = 220
cu

In [202]:
# to be reporoducible
g = torch.Generator().manual_seed(2147483647)
random.seed(10)
# Now we can use our model to predict words!
for i in range(10):
  
  out = ""
  # https://www.nancy.cc/2020/09/10/top-first-letters-of-us-baby-names-2019/
  # Sample from the common starting letters and let's see what we get out.
  letters = "." + random.sample(["a","e","m"],1)[0]
  ixs = chars_to_idx[letters]
  while True:
    # Append letter to name
    out += letters[1]
    x_enc = F.one_hot(torch.tensor([ixs]), num_classes=728).float()
    logits = calc_log_counts(W) 
    counts = calc_counts(logits) 
    p = calc_probs(counts).squeeze()
    # ----------
    # Sample next letter from distribution given by the network!

    ix = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    # update the letters used to predict
    letters = letters[1] + idx_to_char[ix]
    ixs = chars_to_idx[letters]
    # end token -> Stop!
    if ix == 0:
      print(out)
      break

merla
axx
eminavissyn
eu
mariia
aloah
aaniereox
essa
er
ellen


Pretty good :)